## 인터파크티켓 크롤링
#### 공연 개별 상세페이지에서 정보 가져오기

In [ ]:
from selenium import webdriver
import time

path="C:/Users/YYJ/Desktop/myPython/chromedriver.exe"
driver=webdriver.Chrome(path)

In [ ]:
def get_data(ptype):
    
    p_keys=['Type', 'Title', 'Subtitle', 'Venue', 'open_Date', 'end_Date', 'Performer']
    subtitle=venue=o_date=e_date=''
    performer=[]
    
    # Title
    title=driver.find_element_by_id('IDGoodsName').text
    
    elem=driver.find_element_by_id('TabA')
    for dl in elem.find_elements_by_tag_name('dl'):
        info_type=dl.find_element_by_tag_name('dt')
        info_data=dl.find_element_by_tag_name('dd')
        
    # Subtitle, Venue, Date, Performer
        if info_type.text=='장소':
            venue=info_data.text
        elif info_type.text=='기간':
            if '~' in info_data.text:
                o_date=info_data.text.split(' ', 3)[0]       # split('기준', '횟수')
                e_date=info_data.text.split(' ', 3)[2]
            else:
                o_date=e_date=info_data.text.split(' ', 2)[0]
        elif info_type.text=='부제':
            subtitle=info_data.text
        elif info_type.text=='출연':
            driver.find_element_by_id('aTabC').click()
            driver.switch_to.frame('ifrTabC')
            i_actor=driver.find_elements_by_class_name('Actor')
            for i in i_actor:
                performer.append(i.find_element_by_tag_name('h3').text)
        else:
            pass
        
    p_value=[ptype, title, subtitle, venue, o_date, e_date, performer]
    data=dict(zip(p_keys, p_value))
    print(data)
    
    return data

In [ ]:
def get_link():
    
    link=[]
    p_table=driver.find_element_by_xpath('/html/body/table/tbody/tr[2]/td[3]/div/div/div[2]/div/table/tbody')

    for row in p_table.find_elements_by_tag_name('tr'):
        link.append(row.find_element_by_tag_name('a').get_attribute('href'))
        
    return link

In [ ]:
p_type={'Mus':'뮤지컬', 'Liv':'콘서트', 'Dra':'연극', 'Cla':'클래식', 'Eve&SubCa=Eve_T':'전시', 'Fam':'아동'}
p_data=[]       # 공연정보 리스트

for key in p_type:
    
    # [장르 - 전체보기 - 상품명순] 페이지 접근
    driver.get("http://ticket.interpark.com/TPGoodsList.asp?Ca="+key+"&Sort=4")
    time.sleep(0.5)
    
    # 각 공연의 상세페이지 주소 p_link에 가져오기
    p_link=get_link()
    
    # p_link의 주소로 상세페이지에 접근하여 데이터 가져오기
    for link in p_link:
        
        driver.get(link)
        time.sleep(0.5)
            
        # 상세페이지에서 얻은 공연정보 데이터를 p_data에 추가하기
        p_data.append(get_data(p_type[key]))
        